<a href="https://colab.research.google.com/github/ns7123/Xente_Fraud_Detection_Challenge/blob/master/nielsmith_edsa_classif_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Honour Code,<br>
I, Niel Smith, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
%matplotlib inline

In [0]:
import re
import string

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [732]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
definitions = pd.read_csv('/content/drive/My Drive/EDSA 2019/07 Classification/hackathon/Xente_Variable_Definitions.csv')
train = pd.read_csv('/content/drive/My Drive/EDSA 2019/07 Classification/hackathon/training.csv', index_col='TransactionId')
test = pd.read_csv('/content/drive/My Drive/EDSA 2019/07 Classification/hackathon/test.csv', index_col='TransactionId')
example = pd.read_csv('/content/drive/My Drive/EDSA 2019/07 Classification/hackathon/sample_submission.csv', index_col='TransactionId')

In [0]:
# definitions = pd.read_csv('Xente_Variable_Definitions.csv')
# train = pd.read_csv('training.csv', index_col='TransactionId')
# test = pd.read_csv('test.csv', index_col='TransactionId')
# example = pd.read_csv('sample_submission.csv', index_col='TransactionId')

In [735]:
pd.set_option('display.max_colwidth', -1)
definitions

,Column Name,Definition
0,TransactionId,Unique �transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transactions for processing
2,AccountId,Unique number identifying the customer on platform
3,SubscriptionId,Unique number identifying the customer subscription
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item �bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader product categories.


In [736]:
train.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
TransactionId,,,,,,,,,,,,,,,
TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [737]:
test.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
TransactionId,,,,,,,,,,,,,,
TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [738]:
example.head()

,FraudResult
TransactionId,
TransactionId_50600,NaN
TransactionId_95109,NaN
TransactionId_47357,NaN
TransactionId_28185,NaN
TransactionId_22140,NaN


In [739]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95662 entries, TransactionId_76871 to TransactionId_35670
Data columns (total 15 columns):
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null object
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: float64(1), int64(4), object(10)
memory usage: 11.7+ MB


In [0]:
data = pd.concat([train.copy(), test.copy()], sort=False)

In [741]:
data[data['FraudResult']==1]

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
TransactionId,,,,,,,,,,,,,,,
TransactionId_60873,BatchId_93685,AccountId_3832,SubscriptionId_4161,CustomerId_4275,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,700000.0,700000,2018-11-15T08:04:51Z,0,1.0
TransactionId_28016,BatchId_125553,AccountId_2366,SubscriptionId_4834,CustomerId_7414,UGX,256,ProviderId_5,ProductId_9,financial_services,ChannelId_1,725000.0,725000,2018-11-15T08:20:09Z,2,1.0
TransactionId_1149,BatchId_10381,AccountId_3832,SubscriptionId_4161,CustomerId_4275,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,700000.0,700000,2018-11-15T08:31:57Z,0,1.0
TransactionId_136904,BatchId_52644,AccountId_3692,SubscriptionId_3451,CustomerId_4134,UGX,256,ProviderId_1,ProductId_15,financial_services,ChannelId_3,700000.0,700000,2018-11-17T09:15:23Z,0,1.0
TransactionId_88818,BatchId_54481,AccountId_3832,SubscriptionId_4161,CustomerId_4275,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,700000.0,700000,2018-11-17T11:49:46Z,0,1.0
TransactionId_65985,BatchId_34263,AccountId_2366,SubscriptionId_4834,CustomerId_7414,UGX,256,ProviderId_5,ProductId_9,financial_services,ChannelId_1,905000.0,905000,2018-11-19T12:10:31Z,2,1.0
TransactionId_4222,BatchId_127498,AccountId_1959,SubscriptionId_1476,CustomerId_2353,UGX,256,ProviderId_3,ProductId_15,financial_services,ChannelId_3,800000.0,800000,2018-11-20T12:00:29Z,0,1.0
TransactionId_2550,BatchId_79762,AccountId_4003,SubscriptionId_3002,CustomerId_4453,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,770000.0,770000,2018-11-20T12:27:15Z,0,1.0
TransactionId_49480,BatchId_77828,AccountId_3943,SubscriptionId_1417,CustomerId_4391,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,500000.0,500000,2018-11-20T22:11:53Z,0,1.0


In [742]:
data.tail()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
TransactionId,,,,,,,,,,,,,,,
TransactionId_116198,BatchId_117348,AccountId_4841,SubscriptionId_3829,CustomerId_5225,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_3,-5000.0,5000,2019-03-14T23:56:28Z,2,NaN
TransactionId_24829,BatchId_53630,AccountId_4803,SubscriptionId_1738,CustomerId_5272,UGX,256,ProviderId_3,ProductId_10,airtime,ChannelId_3,10000.0,10000,2019-03-14T23:57:38Z,1,NaN
TransactionId_102920,BatchId_112209,AccountId_4817,SubscriptionId_4613,CustomerId_5286,UGX,256,ProviderId_3,ProductId_10,airtime,ChannelId_3,10000.0,10000,2019-03-14T23:59:17Z,1,NaN
TransactionId_128439,BatchId_58734,AccountId_4841,SubscriptionId_3829,CustomerId_5286,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_3,-5000.0,5000,2019-03-14T23:59:39Z,2,NaN
TransactionId_45927,BatchId_98308,AccountId_4593,SubscriptionId_3866,CustomerId_5057,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,3500.0,3500,2019-03-14T23:59:51Z,4,NaN


In [743]:
data['FraudResult'].value_counts()

0.0    95469
1.0    193  
Name: FraudResult, dtype: int64

# Baseline Predict All as Fraud

In [744]:
train['FraudResult'].value_counts()[0]/(train['FraudResult'].value_counts()[0] + train['FraudResult'].value_counts()[1])

0.9979824799816019

In [745]:
metrics.f1_score(train['FraudResult'], np.ones(len(train['FraudResult'])))

0.004026915653852173

In [746]:
print(metrics.classification_report(train['FraudResult'], np.ones(len(train['FraudResult']))))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     95469
           1       0.00      1.00      0.00       193

    accuracy                           0.00     95662
   macro avg       0.00      0.50      0.00     95662
weighted avg       0.00      0.00      0.00     95662



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [747]:
metrics.confusion_matrix(train['FraudResult'], np.ones(len(train['FraudResult'])))

array([[    0, 95469],
       [    0,   193]])

In [748]:
train.shape

(95662, 15)

In [749]:
train.nunique()

BatchId                 94809
AccountId               3633 
SubscriptionId          3627 
CustomerId              3742 
CurrencyCode            1    
CountryCode             1    
ProviderId              6    
ProductId               23   
ProductCategory         9    
ChannelId               4    
Amount                  1676 
Value                   1517 
TransactionStartTime    94556
PricingStrategy         4    
FraudResult             2    
dtype: int64

In [750]:
test.nunique()

BatchId                 44684
AccountId               2471 
SubscriptionId          2470 
CustomerId              5018 
CurrencyCode            1    
CountryCode             1    
ProviderId              6    
ProductId               26   
ProductCategory         9    
ChannelId               5    
Amount                  998  
Value                   944  
TransactionStartTime    44018
PricingStrategy         4    
dtype: int64

# Data Processing

In [751]:
data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
TransactionId,,,,,,,,,,,,,,,
TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0.0
TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0.0
TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0.0
TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0.0
TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0.0


In [0]:
data['TransactionStartTime'] = pd.to_datetime(data['TransactionStartTime'])

In [0]:
data['Time_diff'] = data['TransactionStartTime'].diff().dt.total_seconds()

In [0]:
data['Time_diff'].fillna(data['Time_diff'].mean(), inplace=True)

In [0]:
data['day'] = data['TransactionStartTime'].dt.day
data['month'] = data['TransactionStartTime'].dt.month

In [756]:
data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,Time_diff,day,month
TransactionId,,,,,,,,,,,,,,,,,,
TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0.0,73.639906,15,11
TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0.0,19.000000,15,11
TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0.0,1513.000000,15,11
TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0.0,2914.000000,15,11
TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0.0,86.000000,15,11


In [757]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140681 entries, TransactionId_76871 to TransactionId_45927
Data columns (total 18 columns):
BatchId                 140681 non-null object
AccountId               140681 non-null object
SubscriptionId          140681 non-null object
CustomerId              140681 non-null object
CurrencyCode            140681 non-null object
CountryCode             140681 non-null int64
ProviderId              140681 non-null object
ProductId               140681 non-null object
ProductCategory         140681 non-null object
ChannelId               140681 non-null object
Amount                  140681 non-null float64
Value                   140681 non-null int64
TransactionStartTime    140681 non-null datetime64[ns, UTC]
PricingStrategy         140681 non-null int64
FraudResult             95662 non-null float64
Time_diff               140681 non-null float64
day                     140681 non-null int64
month                   140681 non-null int64
dtypes:

In [0]:
for col in ['CountryCode','PricingStrategy']:
    data[col] = data[col].apply(lambda x: str(x))

In [0]:
data['Value'] = data['Value'].apply(lambda x: float(x))

In [760]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140681 entries, TransactionId_76871 to TransactionId_45927
Data columns (total 18 columns):
BatchId                 140681 non-null object
AccountId               140681 non-null object
SubscriptionId          140681 non-null object
CustomerId              140681 non-null object
CurrencyCode            140681 non-null object
CountryCode             140681 non-null object
ProviderId              140681 non-null object
ProductId               140681 non-null object
ProductCategory         140681 non-null object
ChannelId               140681 non-null object
Amount                  140681 non-null float64
Value                   140681 non-null float64
TransactionStartTime    140681 non-null datetime64[ns, UTC]
PricingStrategy         140681 non-null object
FraudResult             95662 non-null float64
Time_diff               140681 non-null float64
day                     140681 non-null int64
month                   140681 non-null int64
dty

In [761]:
train[train['FraudResult']==1].nunique()

BatchId                 183
AccountId               52 
SubscriptionId          52 
CustomerId              54 
CurrencyCode            1  
CountryCode             1  
ProviderId              5  
ProductId               9  
ProductCategory         4  
ChannelId               3  
Amount                  61 
Value                   61 
TransactionStartTime    193
PricingStrategy         4  
FraudResult             1  
dtype: int64

In [762]:
data.nunique()

BatchId                 139493
AccountId               4841  
SubscriptionId          4836  
CustomerId              7479  
CurrencyCode            1     
CountryCode             1     
ProviderId              6     
ProductId               27    
ProductCategory         10    
ChannelId               5     
Amount                  2099  
Value                   1880  
TransactionStartTime    138574
PricingStrategy         4     
FraudResult             2     
Time_diff               1702  
day                     31    
month                   5     
dtype: int64

In [0]:
cols_keep = ['ProviderId',
             'ProductId',
             'ProductCategory',
             'ChannelId',
             'day',
             'month',
             'Amount',
             'Value',
             'FraudResult',
             'Time_diff']

In [0]:
data = data[cols_keep]

In [765]:
data.isna().sum()

ProviderId         0    
ProductId          0    
ProductCategory    0    
ChannelId          0    
day                0    
month              0    
Amount             0    
Value              0    
FraudResult        45019
Time_diff          0    
dtype: int64

In [766]:
data.head()

,ProviderId,ProductId,ProductCategory,ChannelId,day,month,Amount,Value,FraudResult,Time_diff
TransactionId,,,,,,,,,,
TransactionId_76871,ProviderId_6,ProductId_10,airtime,ChannelId_3,15,11,1000.0,1000.0,0.0,73.639906
TransactionId_73770,ProviderId_4,ProductId_6,financial_services,ChannelId_2,15,11,-20.0,20.0,0.0,19.000000
TransactionId_26203,ProviderId_6,ProductId_1,airtime,ChannelId_3,15,11,500.0,500.0,0.0,1513.000000
TransactionId_380,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,15,11,20000.0,21800.0,0.0,2914.000000
TransactionId_28195,ProviderId_4,ProductId_6,financial_services,ChannelId_2,15,11,-644.0,644.0,0.0,86.000000


In [0]:
def encode_data(input_df, cols):
    
    input_df = pd.get_dummies(input_df, columns=cols, drop_first=True, prefix=cols, prefix_sep='_')
    
    return input_df

In [768]:
cols_keep[:-4]

['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'day', 'month']

In [0]:
data_encoded = encode_data(data, cols_keep[:-4])

In [0]:
data_encoded = data_encoded.drop('FraudResult', axis=1)

In [771]:
data_encoded.head()

,Amount,Value,Time_diff,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,ProductId_ProductId_10,ProductId_ProductId_11,ProductId_ProductId_12,ProductId_ProductId_13,ProductId_ProductId_14,ProductId_ProductId_15,ProductId_ProductId_16,ProductId_ProductId_17,ProductId_ProductId_18,ProductId_ProductId_19,ProductId_ProductId_2,ProductId_ProductId_20,ProductId_ProductId_21,ProductId_ProductId_22,ProductId_ProductId_23,ProductId_ProductId_24,ProductId_ProductId_25,ProductId_ProductId_26,ProductId_ProductId_27,ProductId_ProductId_3,ProductId_ProductId_4,ProductId_ProductId_5,ProductId_ProductId_6,ProductId_ProductId_7,ProductId_ProductId_8,ProductId_ProductId_9,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_retail,ProductCategory_ticket,...,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_4,ChannelId_ChannelId_5,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,month_2,month_3,month_11,month_12
TransactionId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TransactionId_76871,1000.0,1000.0,73.639906,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_73770,-20.0,20.0,19.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_26203,500.0,500.0,1513.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_380,20000.0,21800.0,2914.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_28195,-644.0,644.0,86.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [0]:
data_encoded['Amount'] = data_encoded['Amount'].apply(lambda x: 1 if x >= 0 else 0)

In [773]:
data_encoded.head()

,Amount,Value,Time_diff,ProviderId_ProviderId_2,ProviderId_ProviderId_3,ProviderId_ProviderId_4,ProviderId_ProviderId_5,ProviderId_ProviderId_6,ProductId_ProductId_10,ProductId_ProductId_11,ProductId_ProductId_12,ProductId_ProductId_13,ProductId_ProductId_14,ProductId_ProductId_15,ProductId_ProductId_16,ProductId_ProductId_17,ProductId_ProductId_18,ProductId_ProductId_19,ProductId_ProductId_2,ProductId_ProductId_20,ProductId_ProductId_21,ProductId_ProductId_22,ProductId_ProductId_23,ProductId_ProductId_24,ProductId_ProductId_25,ProductId_ProductId_26,ProductId_ProductId_27,ProductId_ProductId_3,ProductId_ProductId_4,ProductId_ProductId_5,ProductId_ProductId_6,ProductId_ProductId_7,ProductId_ProductId_8,ProductId_ProductId_9,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_retail,ProductCategory_ticket,...,ProductCategory_tv,ProductCategory_utility_bill,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_4,ChannelId_ChannelId_5,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,month_2,month_3,month_11,month_12
TransactionId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TransactionId_76871,1,1000.0,73.639906,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_73770,0,20.0,19.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_26203,1,500.0,1513.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_380,1,21800.0,2914.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
TransactionId_28195,0,644.0,86.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [774]:
data_encoded.isna().sum().sum()

0

In [0]:
scaler = StandardScaler()
# X_scaled = scaler.fit_transform(data_encoded)
X_scaled = data_encoded

# Select X, y

In [0]:
X_sub = X_scaled[len(train):]
X = X_scaled[:len(train)]
y = data['FraudResult'][:len(train)]

In [777]:
X.shape

(95662, 81)

In [778]:
train.shape

(95662, 15)

In [779]:
X_sub.shape

(45019, 81)

In [780]:
y.shape

(95662,)

In [781]:
y.isna().sum()

0

# Basic Model

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [0]:
logR = LogisticRegression(C = 0.1,
                          solver='liblinear',
                          class_weight='balanced')
logR.fit(X_train, y_train)
pred = logR.predict(X_test)

In [784]:
metrics.confusion_matrix(y_test, pred)

array([[18962,   135],
       [    0,    36]])

In [785]:
y_test.sum()

36.0

In [786]:
pred.sum()

171.0

In [787]:
logR.score(X_test, y_test)

0.9929441279464799

In [788]:
print(metrics.classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00     19097
         1.0       0.21      1.00      0.35        36

    accuracy                           0.99     19133
   macro avg       0.61      1.00      0.67     19133
weighted avg       1.00      0.99      1.00     19133



# Grid Search Model

In [0]:
# # Instantiate the logistic model.
# logR = LogisticRegression(solver='saga',
#                           penalty='l2',
#                           n_jobs=-1,
#                           class_weight='balanced',
#                           max_iter=1000)

# # Create the parameters grid dictionary.
# logR_parameters = {'C': [0.001, 0.1, 1.0, 100]}


# f1_scorer = metrics.make_scorer(metrics.f1_score)

# # Set up the grid search.
# logR_grid = GridSearchCV(estimator=logR,
#                          param_grid=logR_parameters,
#                          cv=3,
#                          return_train_score=False,
#                          scoring=f1_scorer)

# logR_grid.fit(X, y)

# logR_grid.best_params_

# Final

In [790]:
logR = LogisticRegression(C = 0.1,
                          solver='liblinear',
                          class_weight='balanced')

logR.fit(X, y)

LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
pred = logR.predict(X_sub)

In [792]:
pred.shape

(45019,)

In [0]:
sub = pd.DataFrame({'FraudResult':pred}, index=test.index)

In [794]:
sub.head()

,FraudResult
TransactionId,
TransactionId_50600,0.0
TransactionId_95109,0.0
TransactionId_47357,0.0
TransactionId_28185,0.0
TransactionId_22140,0.0


In [0]:
sub['FraudResult'] = sub['FraudResult'].apply(lambda x: int(x))

In [796]:
sub['FraudResult'].sum()

506

In [797]:
sub['FraudResult'].shape

(45019,)

In [0]:
# Create a csv output file with the predictions.
sub.to_csv('/content/drive/My Drive/EDSA 2019/07 Classification/hackathon/submission6.csv')